# Data management

## Machine Learning - Classifications

## [Malka Guillot](https://malkaguillot.github.io/)

## HEC Liège | [ECON2306]()

In [1]:
import numpy as np
import pandas as pd
# import patsy

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix 

import seaborn as sns
import matplotlib.pyplot as plt

## Part 1: toy example using a logistic regression

### Load & visualise data

In [ ]:
df=pd.read_csv("../data/beers.csv")
df.shape

In [ ]:
df.head()

In [ ]:
f, ax = plt.subplots(figsize=(7, 5))
sns.countplot(x='is_yummy', data=df)
_ = plt.title('# Yummy vs not yummy')
_ = plt.xlabel('Class (1==Yummy)')

#### Prepare data: split features and labels

In [5]:
# all columns up to the last one:
X = df.iloc[:, :-1]
# only the last column:
y = df.iloc[:, -1]

### Splitting into Training and Test set

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

### Model Building and Training 

#### Creating the pipeline

Before we build the model, 
1. we use the standard scaler function to scale the values into a common range. 
2. Next, we create an instance of LogisticRegression() function for logistic regression.

We are not passing any parameters to `LogisticRegression()` so it will assume default parameters. Some of the important parameters you should know are –

- **penalty**:  It specifies the norm for the penalty
  - Default = L2 

- **C**:  It is the inverse of regularization strength
  - Default = 1.0 

- **solver**: It denotes the optimizer algorithm
  - Default = ‘lbfgs’

We are making use of `Pipeline` to create the model to streamline standard scalar and model building.

In [ ]:
scaler = StandardScaler()

lr = LogisticRegression(max_iter=10000, solver='lbfgs') #syntax if you wand to add hyperparameters

model1 = Pipeline([('standardize', scaler),
                    ('log_reg', lr)])
model1

#### Fit our model to the training data

In [ ]:
model1.fit(X_train, y_train)

In [ ]:
model1.get_params

#### Predictions for the class and for the probabilities

In [ ]:
y_train_hat = model1.predict(X_train) # predicting on the training set
y_train_hat[:10]

In [11]:
y_train_hat_probs = model1.predict_proba(X_train)[:,1] # probabilities of being in class 1

We can see that the model predicts $y_i=1$ when $p_i>0.5$:

In [ ]:
temp= pd.DataFrame({'y_train_hat': y_train_hat, 'y_train_hat_probs': y_train_hat_probs})
temp.head(10)

In [ ]:
temp['y_train_hat_probs'].hist()

#### Performance on the training set

In [ ]:
train_accuracy = accuracy_score(y_train, y_train_hat)*100
train_auc_roc = roc_auc_score(y_train, y_train_hat_probs)*100

print('Confusion matrix:\n', confusion_matrix(y_train, y_train_hat))
print('Training AUC: %.4f %%' % train_auc_roc)
print('Training accuracy: %.4f %%' % train_accuracy)

### Test set

<div class="alert alert-info">
<h4> Your turn</h4>
Following the computation on the train set, compute for the test set:
     
    - the class & proba predictions
    - the accuracy and AUC
</div>

In [ ]:
# Predictions on the test set
y_test_hat = 
y_test_hat_probs = 
# Probabilities of being in class 1

# Metrics


In [ ]:
print(classification_report(y_test, y_test_hat, digits=6))

### Plot the ROC curve

In [ ]:
from sklearn import metrics

fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_hat_probs)
print("tresholds:",  len(threshold))
roc_auc = metrics.auc(fpr, tpr)
roc_auc

In [ ]:
# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

We can further try to improve this model performance by hyperparameter tuning by changing the value of C or choosing other solvers available in `LogisticRegression()`.

## Part 2

The objective is to build a classifier for whether a firm is going to default. 

### Load & visualise data

In [19]:
data = pd.read_csv("../data/bisnode_firms_clean.csv")

In [20]:
#pip install summarytools

In [ ]:
from summarytools import dfSummary
dfSummary(data, is_collapsible = True)

In [ ]:
sns.set_theme()
sns.histplot(data=data, x="default_f", stat="percent") 
plt.title('Default frequency')
plt.xlabel('Default frequency') 
plt.show()

In [23]:
y=data['default']
X=data.drop(columns='default')

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

All sets are balanced:

In [ ]:
print("--- Total ---")
print(y.value_counts(normalize=True))
print("--- Train ---")
print(y_train.value_counts(normalize=True))
print("--- Test ---")
print(y_test.value_counts(normalize=True))

### Model building

There are so many variables !

We are going to compare several models : 
- Logit with a selection of variables **M1**
- Logit with a selection of variables **M2**
- Regularized logit on **M2** variables

Firm related variables

In [26]:
firm = ["age", "age2", "new", "ind2_cat", "m_region_loc", "urban_m"]

Human capital related variables

In [27]:
hr = [
    "female",
    "ceo_age",
    "flag_high_ceo_age",
    "flag_low_ceo_age",
    "flag_miss_ceo_age",
    "ceo_count",
    "labor_avg_mod",
    "flag_miss_labor_avg",
    "foreign_management",
]

Financial variables

In [28]:
qualityvars = ["balsheet_flag", "balsheet_length", "balsheet_notfullyear"]

engvar = [
    "total_assets_bs",
    "fixed_assets_bs",
    "liq_assets_bs",
    "curr_assets_bs",
    "share_eq_bs",
    "subscribed_cap_bs",
    "intang_assets_bs",
    "extra_exp_pl",
    "extra_inc_pl",
    "extra_profit_loss_pl",
    "inc_bef_tax_pl",
    "inventories_pl",
    "material_exp_pl",
    "profit_loss_year_pl",
    "personnel_exp_pl",
]

Growth variables

In [29]:
d1 = [ 
    "d1_sales_mil_log_mod",
    "d1_sales_mil_log_mod_sq",
    "flag_low_d1_sales_mil_log",
    "flag_high_d1_sales_mil_log",
]

In [30]:
M1 = [
    "sales_mil_log",
    "sales_mil_log_sq",
    "d1_sales_mil_log_mod",
    "profit_loss_year_pl",
    "fixed_assets_bs",
    "share_eq_bs",
    "age",
    "foreign_management",
    "ind2_cat",
]
M2 = ["sales_mil_log", "sales_mil_log_sq"] + firm + engvar + d1 + hr

#### Selection of the relevant variables

In [31]:
# M1
X_train_M1=X_train[M1]
X_test_M1 =X_test[M1]

In [32]:
# M2
X_train_M2=X_train[M2]
X_test_M2 =X_test[M2]

#### Set up the method for model selection

In [33]:
k = KFold(n_splits=5, shuffle=True, random_state=42)

#### No regularisation needed so setting the paremeter to very high value

In [34]:
C_value_logit = [1e20]

#### Where we put the results of the different models, for comparison purposes:

In [35]:
test_accuracy={}
test_auc_roc={}

### Model 1: Logit
#### Set up Logit model object

In [36]:
logistic = LogisticRegressionCV(
    Cs=C_value_logit,   #  # Cs: inverse of regularization strength.
    cv=k,               # 5-fold cross-validation
    refit=True,         # Refit the best estimator with the entire dataset
    solver="newton-cg", # Optimization algorithm
    tol=1e-7,           # Tolerance for stopping criteria
    random_state=42,    # Random seed
)

#### Creating the pipeline

In [ ]:
pipeline_logit = Pipeline([
    ('standardize', StandardScaler()),
    ('log_reg', logistic)
                          ])
pipeline_logit

#### On M1 features set

In [ ]:
pipeline_logit.fit(X_train_M1, y_train)

In [ ]:
# Predictions on the test set
y_test_hat = pipeline_logit.predict(X_test_M1)
y_test_hat_probs = pipeline_logit.predict_proba(X_test_M1)[:,1]

# Metrics
test_accuracy['logit_m1'] = accuracy_score(y_test, y_test_hat)*100
test_auc_roc['logit_m1'] = roc_auc_score(y_test, y_test_hat_probs)*100

print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat))
print('Testing AUC: %.4f %%' % test_auc_roc['logit_m1'])
print('Testing accuracy: %.4f %%' % test_accuracy['logit_m1']) 


#### On M2 features set

We need to deal with all the features: depending on their type, the pre-processing will be different

##### Categorical features

In [ ]:
from sklearn.compose import make_column_selector as selector 

categorical_columns_selector = selector(dtype_include=object)  # Selects all columns of type object
categorical_features = categorical_columns_selector(X_train_M2) # Apply the selector to the training set
categorical_features

In [ ]:
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)
categorical_transformer

##### Numeric features

In [ ]:
numeric_features = [col for col in X_train_M2.columns if col not in categorical_features] # Select the columns that are not categorical
len(numeric_features)

In [ ]:
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)
numeric_transformer

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)
preprocessor

##### The pipeline:

In [ ]:
#Append classifier to preprocessing pipeline. Now we have a full prediction pipeline.
pipeline_logit = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", logistic)]
)
pipeline_logit

<div class="alert alert-info">
<h4> Your turn: Fit the pipeline </h4>
Following M1 example, estimate and evaluate a logit model with cross-validation using the M2 set of features.     
</div>

##### Performances for M2

#### Comparing performance for the 2 logit models: 

In [ ]:
test_auc_roc

In [ ]:
test_accuracy

### Model 2: Lasso with standardized X data

In [ ]:
lambdas = list(10 ** np.arange(-1, -4.01, -1 / 3)) # 10^-1, 10^-0.67, 10^-0.33, 10^0, 10^0.33, 10^0.67, 10^1
n_obs = X_train_M2.shape[0] * 4 / 5 # 4/5 of the training set
C_values = [
    1 / (l * n_obs) for l in lambdas
]  # Cs are the inverse of regularization strength
len(C_values)

In [51]:
logistic_lasso = LogisticRegressionCV(
    Cs=C_values,
    penalty="l1", # L1 regularization = lasso 
    cv=k,
    refit=True,
    scoring="roc_auc",
    solver="liblinear",
    random_state=42,
)


<div class="alert alert-info">
<h4> Your turn: Create and implement the pipeline with the logistic lasso </h4>
</div>

In [ ]:
#Append classifier to preprocessing pipeline. Now we have a full prediction pipeline.
pipeline_logistic_lasso = Pipeline(
 
 
pipeline_logistic_lasso

In [54]:
from sklearn.model_selection import cross_val_score

In [55]:
# k = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline_logistic_lasso, X_train_M2, y_train, cv = k)

In [56]:
y_test_hat = pipeline_logistic_lasso.predict(X_test_M2)
y_test_hat_probs = pipeline_logistic_lasso.predict_proba(X_test_M2)[:,1]

In [57]:
test_accuracy['logistic_lasso'] = accuracy_score(y_test, y_test_hat)*100
test_auc_roc['logistic_lasso'] = roc_auc_score(y_test, y_test_hat_probs)*100

In [ ]:
from sklearn import metrics

fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_hat_probs)
roc_auc = metrics.auc(fpr, tpr)
threshold

In [ ]:
# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Comparing the models

In [ ]:
test_auc_roc

In [ ]:
test_accuracy

#### Deciding on the model

`logit_m2` seems to perform marginaly better on the test set. 

### Re-estimating the model on the full dataset

In [ ]:
#Append classifier to preprocessing pipeline. Now we have a full prediction pipeline.
pipeline_logit = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", logistic)]
)
pipeline_logit

In [63]:
X_M2 = X[M2]

In [ ]:
logLasso=pipeline_logit.fit(X_M2, y)
logLasso

### Using the model 

In [65]:
#y_new_hat = logLasso.predict(X_new)

## Going further

In [66]:
# pip install xgboost

Let's look at one of the most powerful machine learning model: `xgboost` (cf. [documentation](https://xgboost.readthedocs.io/en/latest/index.html))

In [67]:
import xgboost as xgb

In [68]:
xgb_model = xgb.XGBClassifier(objective="reg:squarederror", random_state=42)

In [ ]:
#Append classifier to preprocessing pipeline. Now we have a full prediction pipeline.
pipeline_xgp = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", xgb_model)]
)
pipeline_xgp

In [ ]:
model_xgb=pipeline_xgp.fit(X_train_M2, y_train)
model_xgb

In [71]:
y_test_hat = model_xgb.predict(X_test_M2)
y_test_hat_probs = model_xgb.predict_proba(X_test_M2)[:,1]

In [72]:
test_accuracy['xgb'] = accuracy_score(y_test, y_test_hat)*100
test_auc_roc['xgb'] = roc_auc_score(y_test, y_test_hat_probs)*100

In [ ]:
test_accuracy